# IACS Capstone Project Team Spotify 1: Playlist Prediction
### Spotify API dataset creation

In [1]:
import numpy as np
import pandas as pd
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import sys
from sklearn.metrics import mean_squared_error
from math import sqrt

### Authentication

In [2]:
client_credentials_manager = SpotifyClientCredentials(client_id='df846cfd28e745178054587b3484f91c',                                                client_secret='e3d39fc92a954e028ff1490288f3fe5c')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Playlist DataFrame Generation
The below code generates a DataFrame with all the playlists that were generated by the user ID "spotify" - in other words, those created by the company for wide distribution and recommendation. Alongside each playlist ID, we have extracted the mean popularity of the playlist's tracks, the number of followers of the playlist, as well as the playlist's name.

In [3]:
#sp.user_playlists('spotify', limit=50, offset=0)
#playlists

In [4]:
#sp.audio_features(tracks=item['track']['id'])
featured_names = []
featured_playlists = sp.featured_playlists(locale=None, country=None, timestamp=None, limit=50, offset=0)
for i, playlist in enumerate(featured_playlists['playlists']['items']):
    print(playlist['name'])
    featured_names.append(playlist['name'])

Evening Acoustic
electroNOW
Songs For Sleeping
Indie Chillout.
Piano in the Background
Late Night Love
Hit Rewind
9 Hour Sleep
Today's Top Hits
Rage Radio
Jazz For Sleep
Peaceful Piano
Night Rider
Sleep
4am Comedown


In [5]:
featured_names

['Evening Acoustic',
 'electroNOW',
 'Songs For Sleeping',
 'Indie Chillout.',
 'Piano in the Background',
 'Late Night Love',
 'Hit Rewind',
 '9 Hour Sleep',
 "Today's Top Hits",
 'Rage Radio',
 'Jazz For Sleep',
 'Peaceful Piano',
 'Night Rider',
 'Sleep',
 '4am Comedown']

In [6]:
'White Noise' in featured_names

False

In [7]:
import time

class Timer(object):
    def __init__(self, verbose=True):
        self.verbose = verbose

    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        self.secs = self.end - self.start
        self.msecs = self.secs * 1000  # millisecs
        if self.verbose:
            print('elapsed time: %f ms %f projected finish' % (self.msecs, self.msecs*1726/(4*1000*3600)))

In [8]:
with Timer() as t:
    for i in range(1000):
        a = i*34

elapsed time: 0.298977 ms 0.000036 projected finish


78


In [49]:
playlists = sp.user_playlists('spotify', limit=50, offset=(0 + 50*1))
for i, playlist in enumerate(playlists['items']):
    a = sp.user_playlist('spotify', playlist_id=playlist['id'],
                            fields='followers.total,tracks.items(added_at,track.available_markets, track.popularity, track.name, track.id)')
    names = []
    parts = chunks(a['tracks']['items'], 4)
    p1 = parts[0]
    p2 = parts[1]
    p3 = parts[2]
    p4 = parts[3]

    for item in p1:
        names.append(item['track']['name'])
    for item in p2:
        names.append(item['track']['name'])
    for item in p3:
        names.append(item['track']['name'])
    for item in p4:
        names.append(item['track']['name'])

In [38]:
def chunks(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0
    
    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
        
    return out

In [54]:
def getPlaylistData(offset, numLoops, queue, limit=50):
    names = []
    total_tracks = []
    followers = []
    popularity_means = []
    mean_num_markets = []
    
    added_date = []

    q1_audio_attributes_median = []
    q2_audio_attributes_median = []
    q3_audio_attributes_median = []
    q4_audio_attributes_median = []
    
    featured = []
    #audio_attributes = np.empty((numLoops*limit,12), dtype=np.float64)
    playlist_ids = []

    for loopIndex in range(numLoops):
        playlists = sp.user_playlists('spotify', limit=limit, offset=(offset + limit*loopIndex))
        for i, playlist in enumerate(playlists['items']):
            try:
                with Timer() as t:
                    metadata = sp.user_playlist('spotify', playlist_id=playlist['id'],
                        fields='followers.total,tracks.items(added_at,track.available_markets, track.popularity, track.name, track.id)')
                    
                    parts = chunks(metadata['tracks']['items'], 4)
                    
                    # PART 1
                    
                    p1 = parts[0]
                    
                    q1_popularities = np.empty(len(p1))
                    #q1_added_at = []
                    q1_num_markets = np.empty(len(p1))
                    q1_aud_features = np.empty((len(p1), 12))
                    q1_ids = []

                    for ix, item in enumerate(p1):
                        
                        q1_ids.append(item['track']['id'])
                        q1_popularities[ix] = item['track']['popularity']
                        q1_added_at.append(item['added_at'])
                        q1_num_markets[ix] = len(item['track']['available_markets'])
                        
                    q1_features = sp.audio_features(tracks=q1_ids)
                        
                    for index, songData in enumerate(q1_features):
                        q1_aud_features[index][0] = songData['acousticness']
                        q1_aud_features[index][1] = songData['danceability']
                        q1_aud_features[index][2] = songData['energy']
                        q1_aud_features[index][3] = songData['instrumentalness']
                        q1_aud_features[index][4] = songData['key']
                        q1_aud_features[index][5] = songData['liveness']
                        q1_aud_features[index][6] = songData['loudness']
                        q1_aud_features[index][7] = songData['mode']
                        q1_aud_features[index][8] = songData['tempo']
                        q1_aud_features[index][9] = songData['time_signature']
                        q1_aud_features[index][10] = songData['valence']
                        q1_aud_features[index][11] = songData['duration_ms']
                    
                    q1_audio_attributes_median.append(np.percentile(q1_aud_features,50, axis=0))
                    q1_popularity_means.append(q1_popularities.mean())

                    #added_date.append(q1_added_at[0])
                    q1_mean_num_markets.append(q1_num_markets.mean())
                    q1_popularity_means.append(q1_popularities.mean())
                    
                    # PART 2
                    
                    p2 = parts[1]
                    
                    q2_popularities = np.empty(len(p2))
                    #q2_added_at = []
                    q2_num_markets = np.empty(len(p2))
                    q2_aud_features = np.empty((len(p2), 12))
                    q2_ids = []

                    for ix, item in enumerate(p2):
                        
                        q2_ids.append(item['track']['id'])
                        q2_popularities[ix] = item['track']['popularity']
                        q2_added_at.append(item['added_at'])
                        q2_num_markets[ix] = len(item['track']['available_markets'])
                        
                    q2_features = sp.audio_features(tracks=q2_ids)
                        
                    for index, songData in enumerate(q2_features):
                        q2_aud_features[index][0] = songData['acousticness']
                        q2_aud_features[index][1] = songData['danceability']
                        q2_aud_features[index][2] = songData['energy']
                        q2_aud_features[index][3] = songData['instrumentalness']
                        q2_aud_features[index][4] = songData['key']
                        q2_aud_features[index][5] = songData['liveness']
                        q2_aud_features[index][6] = songData['loudness']
                        q2_aud_features[index][7] = songData['mode']
                        q2_aud_features[index][8] = songData['tempo']
                        q2_aud_features[index][9] = songData['time_signature']
                        q2_aud_features[index][10] = songData['valence']
                        q2_aud_features[index][11] = songData['duration_ms']
                    
                    q2_audio_attributes_median.append(np.percentile(q2_aud_features,50, axis=0))
                    q2_popularity_means.append(q4_popularities.mean())
                    
                    #added_date.append(q2_added_at[0])
                    q2_mean_num_markets.append(q2_num_markets.mean())
                    q2_popularity_means.append(q2_popularities.mean())
                    
                    # PART 3
                    
                    p3 = parts[2]
                    
                    q3_popularities = np.empty(len(p3))
                    q3_num_markets = np.empty(len(p3))
                    q3_aud_features = np.empty((len(p3), 12))
                    q3_ids = []

                    for ix, item in enumerate(p3):
                        
                        q3_ids.append(item['track']['id'])
                        q3_popularities[ix] = item['track']['popularity']
                        q3_added_at.append(item['added_at'])
                        q3_num_markets[ix] = len(item['track']['available_markets'])
                        
                    q3_features = sp.audio_features(tracks=q3_ids)
                        
                    for index, songData in enumerate(q3_features):
                        q3_aud_features[index][0] = songData['acousticness']
                        q3_aud_features[index][1] = songData['danceability']
                        q3_aud_features[index][2] = songData['energy']
                        q3_aud_features[index][3] = songData['instrumentalness']
                        q3_aud_features[index][4] = songData['key']
                        q3_aud_features[index][5] = songData['liveness']
                        q3_aud_features[index][6] = songData['loudness']
                        q3_aud_features[index][7] = songData['mode']
                        q3_aud_features[index][8] = songData['tempo']
                        q3_aud_features[index][9] = songData['time_signature']
                        q3_aud_features[index][10] = songData['valence']
                        q3_aud_features[index][11] = songData['duration_ms']
                    
                    q3_audio_attributes_median.append(np.percentile(q3_aud_features,50, axis=0))
                    q3_popularity_means.append(q3_popularities.mean())

                    #added_date.append(q3_added_at[0])
                    q3_mean_num_markets.append(q3_num_markets.mean())
                    
                    # PART 4
                    
                    p4 = parts[3]
                    
                    q4_popularities = np.empty(len(p4))
                    #q4_added_at = []
                    q4_num_markets = np.empty(len(p4))
                    q4_aud_features = np.empty((len(p4), 12))
                    q4_ids = []

                    for ix, item in enumerate(p4):
                        
                        q4_ids.append(item['track']['id'])
                        q4_popularities[ix] = item['track']['popularity']
                        q4_added_at.append(item['added_at'])
                        q4_num_markets[ix] = len(item['track']['available_markets'])
                        
                    q4_features = sp.audio_features(tracks=q4_ids)
                        
                    for index, songData in enumerate(q4_features):
                        q4_aud_features[index][0] = songData['acousticness']
                        q4_aud_features[index][1] = songData['danceability']
                        q4_aud_features[index][2] = songData['energy']
                        q4_aud_features[index][3] = songData['instrumentalness']
                        q4_aud_features[index][4] = songData['key']
                        q4_aud_features[index][5] = songData['liveness']
                        q4_aud_features[index][6] = songData['loudness']
                        q4_aud_features[index][7] = songData['mode']
                        q4_aud_features[index][8] = songData['tempo']
                        q4_aud_features[index][9] = songData['time_signature']
                        q4_aud_features[index][10] = songData['valence']
                        q4_aud_features[index][11] = songData['duration_ms']
                    
                    q4_audio_attributes_median.append(np.percentile(q4_aud_features,50, axis=0))
                    q4_popularity_means.append(q4_popularities.mean())
                    q4_mean_num_markets.append(q4_num_markets.mean())

                    #added_date.append(q4_added_at[0])
                    
                    # SUMMARY
                    
                    playlist_ids.append(playlist['id'])
                    followers.append(metadata['followers']['total'])
                    names.append(playlist['name'])
                    featured.append(playlist['name'] in featured_names)
                    total_tracks.append(playlist['tracks']['total'])
                    
                    print("%4d %s" % (i + 1 + playlists['offset'], playlist['name']))
            except:
            #    a=0
                print("NO METADATA")
        if playlists['next']==None:
            break
    
    q1_audio_attributes_median = np.array(q1_audio_attributes_median)
    q2_audio_attributes_median = np.array(q2_audio_attributes_median)
    q3_audio_attributes_median = np.array(q3_audio_attributes_median)
    q4_audio_attributes_median = np.array(q4_audio_attributes_median)

    playlist_data = pd.DataFrame({
    'followers': followers,
    'names': names,
    'playlist_id': playlist_ids,
    'total_tracks': total_tracks,
    'featured': featured,
    #'added_date': added_date, 
            
    # Q1 
    'q1_num_markets': q1_mean_num_markets,
    'q1_mean_popularity': q1_popularity_means,
    'q1_acousticness': q1_audio_attributes_median[:,0],
    'q1_danceability': q1_audio_attributes_median[:,1],
    'q1_energy': q1_audio_attributes_median[:,2],
    'q1_instrumentalness': q1_audio_attributes_median[:,3],
    'q1_key': q1_audio_attributes_median[:,4],
    'q1_liveness_q1': q1_audio_attributes_median[:,5],
    'q1_loudness': q1_audio_attributes_median[:,6],
    'q1_mode': q1_audio_attributes_median[:,7],
    'q1_tempo': q1_audio_attributes_median[:,8],
    'q1_time_signature': q1_audio_attributes_median[:,9],
    'q1_valence': q1_audio_attributes_median[:,10],
    'q1_duration': q1_audio_attributes_median[:,11],
            
    # Q2
            
    'q2_num_markets': q2_mean_num_markets,     
    'q2_mean_popularity': q2_popularity_means,
    'q2_acousticness': q2_audio_attributes_median[:,0],
    'q2_danceability': q2_audio_attributes_median[:,1],
    'q2_energy': q2_audio_attributes_median[:,2],
    'q2_instrumentalness': q2_audio_attributes_median[:,3],
    'q2_key': q2_audio_attributes_median[:,4],
    'q2_liveness_q1': q2_audio_attributes_median[:,5],
    'q2_loudness': q2_audio_attributes_median[:,6],
    'q2_mode': q2_audio_attributes_median[:,7],
    'q2_tempo': q2_audio_attributes_median[:,8],
    'q2_time_signature': q2_audio_attributes_median[:,9],
    'q2_valence': q2_audio_attributes_median[:,10],
    'q2_duration': q2_audio_attributes_median[:,11],
         
    # Q3

    'q3_num_markets': q3_mean_num_markets,            
    'q3_mean_popularity': q3_popularity_means,
    'q3_acousticness': q3_audio_attributes_median[:,0],
    'q3_danceability': q3_audio_attributes_median[:,1],
    'q3_energy': q3_audio_attributes_median[:,2],
    'q3_instrumentalness': q3_audio_attributes_median[:,3],
    'q3_key': q3_audio_attributes_median[:,4],
    'q3_liveness_q1': q3_audio_attributes_median[:,5],
    'q3_loudness': q3_audio_attributes_median[:,6],
    'q3_mode': q3_audio_attributes_median[:,7],
    'q3_tempo': q3_audio_attributes_median[:,8],
    'q3_time_signature': q3_audio_attributes_median[:,9],
    'q3_valence': q3_audio_attributes_median[:,10],
    'q3_duration': q3_audio_attributes_median[:,11],
            
    # Q4 

    'q4_num_markets': q4_mean_num_markets,
    'q4_mean_popularity': q4_popularity_means,
    'q4_acousticness': q4_audio_attributes_median[:,0],
    'q4_danceability': q4_audio_attributes_median[:,1],
    'q4_energy': q4_audio_attributes_median[:,2],
    'q4_instrumentalness': q4_audio_attributes_median[:,3],
    'q4_key': q4_audio_attributes_median[:,4],
    'q4_liveness_q1': q4_audio_attributes_median[:,5],
    'q4_loudness': q4_audio_attributes_median[:,6],
    'q4_mode': q4_audio_attributes_median[:,7],
    'q4_tempo': q4_audio_attributes_median[:,8],
    'q4_time_signature': q4_audio_attributes_median[:,9],
    'q4_valence': q4_audio_attributes_median[:,10],
    'q4_duration': q4_audio_attributes_median[:,11],

    })
    
    q.put(playlist_data)
    name = multiprocessing.current_process().name
    print(name, 'Exiting')

In [56]:
from multiprocessing import Process, Queue

if __name__ == '__main__':
    q = Queue()
    for offset in [0,400,800,1200,1600]:
    #for offset in [0]:

        Process(target=getPlaylistData, args=(offset,8,q)).start()
    #p.start()
    #print(q.get())    # prints "[42, None, 'hello']"
    #p.join()
    

In [ ]:
all_dfs = []
for _ in range(5):
    all_dfs.append(q.get())

In [12]:
playlist_data = all_dfs[0]
for each in all_dfs[1:]:
    playlist_data = playlist_data.append(each,ignore_index=True)

In [3]:
playlist_data

NameError: name 'playlist_data' is not defined

In [14]:
playlist_data.to_csv("playlist_data_with_audio_attributes_percentiles_3.11.17.csv")

In [16]:
sp.user_playlist('spotify','5FJXhjdILmRA2z5bvz4nzf')['tracks']['items'][0]

{'added_at': '2017-03-10T17:25:58Z',
 'added_by': {'external_urls': {'spotify': 'http://open.spotify.com/user/spotify'},
  'href': 'https://api.spotify.com/v1/users/spotify',
  'id': 'spotify',
  'type': 'user',
  'uri': 'spotify:user:spotify'},
 'is_local': False,
 'track': {'album': {'album_type': 'single',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2qxJFvFYMEDqd7ui6kSAcq'},
     'href': 'https://api.spotify.com/v1/artists/2qxJFvFYMEDqd7ui6kSAcq',
     'id': '2qxJFvFYMEDqd7ui6kSAcq',
     'name': 'Zedd',
     'type': 'artist',
     'uri': 'spotify:artist:2qxJFvFYMEDqd7ui6kSAcq'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/2wUjUUtkb5lvLKcGKsKqsR'},
     'href': 'https://api.spotify.com/v1/artists/2wUjUUtkb5lvLKcGKsKqsR',
     'id': '2wUjUUtkb5lvLKcGKsKqsR',
     'name': 'Alessia Cara',
     'type': 'artist',
     'uri': 'spotify:artist:2wUjUUtkb5lvLKcGKsKqsR'}],
   'available_markets': ['CA', 'MX', 'US'],
   'external_urls': {

In [79]:
playlist_sample = sp.user_playlist('spotify','5FJXhjdILmRA2z5bvz4nzf')['tracks']['items']
list_of_tracks = []
for a in playlist_sample:
    list_of_tracks.append(a['track'])
sample = pd.DataFrame(list_of_tracks)[['id','name','external_ids','artists',
                                       'duration_ms','explicit','track_number','popularity']]
features = sp.audio_features(tracks=sample['id'])
features_df = pd.DataFrame(features)
sample['acousticness'] = features_df['acousticness']
sample['danceability'] = features_df['danceability']
sample['energy'] = features_df['energy']
sample['instrumentalness'] = features_df['instrumentalness']
sample['key'] = features_df['key']
sample['liveness'] = features_df['liveness']
sample['loudness'] = features_df['loudness']
sample['mode'] = features_df['speechiness']
sample['tempo'] = features_df['tempo']
sample['time_signature'] = features_df['time_signature']
sample['valence'] = features_df['valence']
sample['sequence'] = sample.index + 1
sample.head()

,id,name,external_ids,artists,duration_ms,explicit,track_number,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,tempo,time_signature,valence,sequence
0,1dNIEtp7AY3oDAKCGg2XkH,Something Just Like This,{u'isrc': u'USQX91700278'},"[{u'name': u'The Chainsmokers', u'external_url...",247626,False,1,0,0.0306,0.607,0.649,0.000025,11,0.1740,-6.695,0.0362,102.996,4,0.470,1
1,12GEpg2XOPyqk03JZEZnJs,It Ain’t Me (with Selena Gomez),{u'isrc': u'SEBGA1700015'},"[{u'name': u'Kygo', u'external_urls': {u'spoti...",220780,False,1,76,0.0905,0.648,0.532,0.000000,0,0.0831,-6.597,0.0746,99.983,4,0.497,2
2,6AeQlMyRzvSl1nkFztZyKl,Issues,{u'isrc': u'USUM71615691'},"[{u'name': u'Julia Michaels', u'external_urls'...",176346,False,1,82,0.4160,0.704,0.423,0.000000,8,0.0607,-6.792,0.0862,113.962,4,0.450,3
3,0FE9t6xYkqWXU2ahLh6D8X,Shape of You,{u'isrc': u'GBAHS1600463'},"[{u'name': u'Ed Sheeran', u'external_urls': {u...",233712,False,1,100,0.5810,0.825,0.652,0.000000,1,0.0931,-3.183,0.0802,95.977,4,0.933,4
4,3ebXMykcMXOcLeJ9xZ17XH,Scared To Be Lonely,{u'isrc': u'NLM5S1600025'},"[{u'name': u'Martin Garrix', u'external_urls':...",220883,False,1,91,0.0895,0.584,0.540,0.000000,1,0.2610,-7.786,0.0576,137.972,4,0.190,5
